In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125352")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

ModuleNotFoundError: No module named 'azureml'

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "Standard_D2_V2"
compute_cluster_name = "udacity-cc"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Compute cluster found.")
except Exception as e:
    print("Creating compute cluster...")
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
        max_nodes=4)
    compute_target = ComputeTarget.create(workspace=ws, name=compute_cluster_name, 
        provisioning_configuration=compute_cluster_config)
    compute_target.wait_for_completion(show_output=True)

ModuleNotFoundError: No module named 'azureml'

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.5, 2),
    "max_iter": choice(50, 100, 250)
})### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=2)### YOUR CODE HERE ###

code_dir = "./training"
if "training" not in os.listdir():
    os.mkdir(code_dir)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", 
    compute_target=compute_target, entry_script="train.py")### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=25,
    max_concurrent_runs=8)### YOUR CODE HERE ###

ModuleNotFoundError: No module named 'azureml'

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

NameError: name 'exp' is not defined

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())

NameError: name 'hyperdrive_run' is not defined

In [6]:
best_run.get_file_names()

NameError: name 'best_run' is not defined

In [7]:
best_run.download_file(name="outputs/model.joblib", output_file_path="./outputs/")

NameError: name 'best_run' is not defined

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

file_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=file_url)

ModuleNotFoundError: No module named 'azureml'

In [9]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x["y"] = y

train, test = train_test_split(x, test_size=0.2, random_state=42)

if not os.path.isdir('input'):
    os.mkdir('input')

# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(train).to_csv("./input/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./input', target_path='input', overwrite=True, show_progress=True)

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('input/train_data.csv'))
label = "y"

ModuleNotFoundError: No module named 'train'

In [10]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5)

ModuleNotFoundError: No module named 'azureml'

In [11]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

automl_run = exp.submit(automl_config, show_output = True)
RunDetails(automl_run).show()

ModuleNotFoundError: No module named 'azureml'

In [12]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_automl_model = automl_run.get_output()
print(best_automl_run)

NameError: name 'automl_run' is not defined

In [13]:
print(best_automl_run.get_metrics())

NameError: name 'best_automl_run' is not defined

In [14]:
best_automl_model[1].estimators

NameError: name 'best_automl_model' is not defined

In [15]:
joblib.dump(best_automl_model, 'outputs/automl_model.joblib')

NameError: name 'best_automl_model' is not defined

In [16]:
compute_target.delete()

NameError: name 'compute_target' is not defined